# Simulation

Now we can start introducing the simulation component of the digital twin. Beside the digital twin state model, we need to introduce the agent model before we can execute the simulation.

<img src="../docs/assets/imgs/modelling_variants.png" alt="Modelling Variants" width="600" align="center">

The OFacT use a multi-agent simulation framework to control production and logistics systems and an event discrete simulation approach to change the digital twin state based on the decisions made by the agents.

Therefore, we need the already introduced *digital twin state model* and the *agent model* to execute the simulation.


## Digital Twin State Model

The *digital twin state model* can be without dynamics (process executions) or with dynamics in the past, but should include an order pool with orders not finished. These orders are the basis to derive processes to be executed in the simulation and change the digital twin.


In [ ]:
from ofact.planning_services.model_generation.persistence import get_state_model_file_path, deserialize_state_model
from ofact.twin.repository_services.deserialization.order_types import OrderType
from projects.tutorial.settings import PROJECT_PATH

# file name of the digital twin state model
digital_twin_state_model_file_name = "input_model.xlsx"

# creating the path to the digital twin state model stored
path_to_model = "models/twin/"
state_model_file_path = get_state_model_file_path(project_path=PROJECT_PATH,
                                                  state_model_file_name=digital_twin_state_model_file_name,
                                                  path_to_model=path_to_model)

# model generation settings (used when the state model is stored in Excel)
state_model_generation_settings = {"order_generation_from_excel": False,
                                   "customer_generation_from_excel": True,
                                   "customer_amount": 5, "order_amount": 20,
                                   "order_type": OrderType.PRODUCT_CONFIGURATOR}

# get the state model from Excel
state_model = deserialize_state_model(state_model_file_path, persistence_format="xlsx",
                                      state_model_generation_settings=state_model_generation_settings,
                                      deserialization_required=False)

# Alternatively
# get state model from Pickle
state_model_pkl = deserialize_state_model(state_model_file_path, persistence_format="pkl")

## Agent Model

For the multi-agent based simulation, we need to introduce the agent model.
The agent model contains the agents, negotiating with each other to get a local optimized solution.

In general with differentiate different agent types:
- Order Pool Agent: is responsible for the order pool and can be requested to release an order to the system
- Order Agent: is responsible to fulfill the features of the order by negotiating the execution of derived processes
- Information Service Agent: is responsible to provide requested information such as "currently no resources able to execute a process"

- Resource Agent: is responsible for one or more resources that should be occupied
    - WorkStation Agent: is mainly responsible for workstation resources
    - Warehouse Agent: is mainly responsible for resources that store/ hold parts or resources
    - Transport Agent: is mainly responsible for resources that can move parts or resources
- Coordinator Agent: is responsible to coordinate processes especially for more than one resource

The main idea is that the order and resource agents negotiate with each other to fulfill the orders.

Columns to specify:

| Column/ Parameter Name | Description                                                                                                                                                                                                                       |
|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| amount                 | number of parallel existing agents with the same attributes (only the name would be individualized)                                                                                                                               |
| name | name of the agent                                                                                                                                                                                                                 |
| agents | agents organization, on which all agents have access to (relevant e.g., for the round handling)                                                                                                                                   | # ToDo: rename to "organization""
| password_xmpp_server | agent password (standard value from settings)
| ip_address_xmpp_server | agent ip address (standard value from settings)
| digital_twin | digital twin state model object                                                                                                                                                                                                   | # ToDo: rename to "state model"
| change_handler | (to_be_defined) if required for the agent, the change handler is placed also in the agent (bool value)                                                                                                                            |
| address_book | dict of contacts (key: resource or entity name, value: agent name) the order agent has addtionally the contact of the order pool agent and the information service agent as well as the resources agents to the coordinator agent |
| processes	|                                                                                                                                                                                                                                   |
| resources	| list of resources, the agent is responsible for                                                                                                                                                                                   |
| preferences | complementary list of preferences associated with the resources                                                                                                                                                                   |
| possible_processes | (to_be_defined) derived from the list resources and the available processes in the digitial twin state model                                                                                                                      |  # ToDo: derivation in the resource agent from the resources itself
| entity_type_to_store |                                                                                                                                                                                           |  # ToDo handle dynamically incoming goods
| entity_types_to_store |                                                                                                                                                                                                                               |  # ToDo handle dynamically incoming goods
| process_provider | (to_be_defined)                                                                                                                                                                                                                   |
| entity_provider | (to_be_defined)                                                                                                                                                                                                                   |
| transport_provider | (to_be_defined)                                                                                                                                                                                                                   |
| value_added_processes	| (to_be_defined) for the order agent to derive the value added processes from the features chosen by the orders                                                                                                                    |
| order_pool | (to_be_defined) derived from the state model                                                                                                                                                                                      |

Preferences

Each resource get a preference that contains reference objects and the accepted time horizont.
| Column/ Parameter Name | Description                         |
|------------------------|-------------------------------------|
| reference_objects      | in general a list of one resource   |
| accepted_time_horizont | time horizont for the planning task |

In [ ]:
from datetime import datetime
from pathlib import Path
from projects.bicycle_world.twin.agent_control.simulate import SimulationStarterBicycleWorld
from projects.tutorial.settings import (PROJECT_PATH, XMPP_SERVER_IP_ADDRESS, XMPP_SERVER_REST_API_PORT,
                                        XMPP_SERVER_SHARED_SECRET, XMPP_SERVER_REST_API_USERS_ENDPOINT)

In [ ]:
# simulation starter object that stores the general information such as the xmpp server data
simulation_starter = (
    SimulationStarterBicycleWorld(project_path=PROJECT_PATH, path_to_models="",
                                  xmpp_server_ip_address=XMPP_SERVER_IP_ADDRESS,
                                  xmpp_server_rest_api_port=XMPP_SERVER_REST_API_PORT,
                                  xmpp_server_shared_secret=XMPP_SERVER_SHARED_SECRET,
                                  xmpp_server_rest_api_users_endpoint=XMPP_SERVER_REST_API_USERS_ENDPOINT))
# file name of the model
agents_model_file_name = "board_game.xlsx"

# number of order agents (each order agent is responsible for one order - (sequential 1-1 relation))
order_agent_amount = 1

# here you can introduce paths to update the digital twin state model
# e.g., an Excel modeled schedule for the resources that updates their intern (state model) schedules (of the resources)
digital_twin_update_paths = {}  # {"resource_schedule": "settings.xlsx"}

# path where the resulting state model (including the dynamics) is stored
digital_twin_state_model_result_path = Path(str(PROJECT_PATH), "scenarios/current/results/six_orders.pkl")

# method that execute the simulation
simulation_starter.simulate(digital_twin=state_model,
                            start_time_simulation=datetime.now(),
                            digital_twin_update_paths=digital_twin_update_paths,
                            agents_file_name=agents_model_file_name,
                            order_agent_amount=order_agent_amount,
                            digital_twin_result_path=digital_twin_state_model_result_path)

Knowing the basics for the simulation, you can execute the board game and the bicycle factory simulation.
As you have learned in the second part,

You can also try to change the number of order agents or try to change the state model ...

Note: For the simulation, a xmpp server is required:
- Reference to the guide to install the xmpp Server: docs/dev/setup_guide.md